In [1]:
# Don't change the style of these imports.
# They must be imported this way for parallelization
# to work correctly.
import lxml.html
import ujson
import gzip
import csv
import datetime

In [2]:
from pdb import set_trace as debug
from IPython.parallel import Client,require
from collections import Counter
from pandas.io.parsers import read_csv

In [3]:
#--> RAW_FILE_DIR with the directory that contains google_play__main.json.gz
#--> OUT_DIR with the directory of where you want the processed files to go
#--> DEBUG whether to print out created CSV files or not
#--> COMPRESS_LEVEL gzip compression level. warning: severely impacts runtime.
#--> LINE_LIMIT how many lines to iterate through in the raw file. For debugging.
DUMP_DATE = '2015_03_21_23_12'
RAW_FILE_DIR = '/Users/kushagrasharma/Coding/data'
OUT_DIR = '/Users/kushagrasharma/Coding/data/out'
DEBUG = 1
COMPRESS_LEVEL = 1
LINE_LIMIT = -1 # -1 means run for the entire file
PARALLEL = 0

In [4]:
# Don't change these!
RAW_FILE_IPHONE = '{}/test_appannie__iphone.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IPAD = '{}/test_appannie__ipad.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_GOOGLE = '{}/test_appannie__google.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_HISTORICALRANKINGS = '{}/appannie__historical_rankings.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_HISTORICALEVENTS = '{}/appannie__historical_events.json.gz'.format(RAW_FILE_DIR)

OUT_BASE = OUT_DIR+'/{}__'+DUMP_DATE+'.csv.gz'

In [5]:
# ! zcat "$RAW_FILE_IPHONE" | head -n1
# ! zcat "$RAW_FILE_IPAD" | head -n1
# ! zcat "$RAW_FILE_GOOGLE" | head -n1

In [6]:
def iter_json_gzip(filename,LINE_LIMIT=LINE_LIMIT,no_verify_appid_time=1):
    with gzip.open(filename,'rt') as file_iter:
        c = 0
        for line in file_iter:
            c +=1
            if c > LINE_LIMIT and LINE_LIMIT>0:
                break
            if isinstance(line,str):
                if len(line)>0:        
                    out = ujson.loads(line)
                    if isinstance(out,dict):
                        if 'app_id' in out and 'timestamp' in out \
                        or no_verify_appid_time:
                            yield out

In [7]:
if PARALLEL:
    ipython_parallel = Client()
    print("{} active computing engines".format(len(ipython_parallel.ids)))

    lbv = ipython_parallel.load_balanced_view()

    map = lambda f,itertable:lbv.map(f,itertable,\
    block =False,\
    ordered =False)

    @require('socket')
    def host(dummy):
        return socket.gethostname()
    
    nodes = list(lbv.map(host,ipython_parallel.ids))
    nodes = [int(x.split('equity')[1].split(".")[0]) for x in nodes]
    nodes = nodes

    print(sorted(list(Counter(nodes).items())))

# RANKINGS_IPHONE

In [8]:
@require(iter_json_gzip,'ujson','gzip','csv','lxml.html')
def RANKINGS_IPHONE(file_in,out_base,compress=1):
    headers = ['app_id', 'chart', 'date', 'country', 'rank']
    out_filename = out_base.format('RANKINGS_IPHONE')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            tree = lxml.html.fromstring(json['html'])
            app_ids = tree.xpath("//span[@style='display:none']")
            chart = ['free', 'paid', 'grossing']
            for i,app in enumerate(app_ids):
                app_info = [app.text_content(), chart[i%3], json['date'], 'United States', int(((i/3)+1))]
                csv_writer.writerow(app_info)
    return out_filename

In [9]:
df_debug = None
if DEBUG:
    df_debug = read_csv(RANKINGS_IPHONE(RAW_FILE_IPHONE,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,chart,date,country,rank
0,826459257,free,2014-03-01,United States,1
1,503190232,paid,2014-03-01,United States,1
2,529479190,grossing,2014-03-01,United States,1
3,808176012,free,2014-03-01,United States,2
4,479516143,paid,2014-03-01,United States,2
5,553834731,grossing,2014-03-01,United States,2
6,766894692,free,2014-03-01,United States,3
7,623592465,paid,2014-03-01,United States,3
8,667728512,grossing,2014-03-01,United States,3
9,825289439,free,2014-03-01,United States,4


# RANKINGS_GOOGLE

In [10]:
@require(iter_json_gzip,'ujson','gzip','csv','lxml.html')
def RANKINGS_GOOGLE(file_in,out_base,compress=1):
    headers = ['app_id', 'chart', 'date', 'country', 'rank']
    out_filename = out_base.format('RANKINGS_GOOGLE')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            tree = lxml.html.fromstring(json['html'])
            app_ids = tree.xpath("//span[@style='display:none']")
            chart = ['free', 'paid', 'grossing', 'new_free', 'new_paid']
            for i,app in enumerate(app_ids):
                app_info = [app.text_content(), chart[i%5], json['date'], 'United States', int(((i/5)+1))]
                csv_writer.writerow(app_info)
    return out_filename

In [11]:
df_debug = None
if DEBUG:
    df_debug = read_csv(RANKINGS_GOOGLE(RAW_FILE_GOOGLE,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,chart,date,country,rank
0,com.facebook.katana,free,2014-03-01,United States,1
1,com.mojang.minecraftpe,paid,2014-03-01,United States,1
2,com.supercell.clashofclans,grossing,2014-03-01,United States,1
3,it.junglestudios.splashyfish,new_free,2014-03-01,United States,1
4,com.FireproofStudios.TheRoom2,new_paid,2014-03-01,United States,1
5,com.pandora.android,free,2014-03-01,United States,2
6,com.touchtype.swiftkey,paid,2014-03-01,United States,2
7,com.king.candycrushsaga,grossing,2014-03-01,United States,2
8,com.square_enix.android_googleplay.deadmanscro...,new_free,2014-03-01,United States,2
9,com.mtvn.sbmigoogleplay,new_paid,2014-03-01,United States,2


# RANKINGS_IPAD

In [12]:
@require(iter_json_gzip,'ujson','gzip','csv','lxml.html')
def RANKINGS_IPAD(file_in,out_base,compress=1):
    headers = ['app_id', 'chart', 'date', 'country', 'rank']
    out_filename = out_base.format('RANKINGS_IPAD')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            tree = lxml.html.fromstring(json['html'])
            app_ids = tree.xpath("//span[@style='display:none']")
            chart = ['free', 'paid', 'grossing']
            for i,app in enumerate(app_ids):
                app_info = [app.text_content(), chart[i%3], json['date'], 'United States', int(((i/3)+1))]
                csv_writer.writerow(app_info)
    return out_filename

In [13]:
df_debug = None
if DEBUG:
    print(type(RANKINGS_IPAD(RAW_FILE_IPAD,OUT_BASE,COMPRESS_LEVEL)))
    df_debug = read_csv(RANKINGS_IPAD(RAW_FILE_IPAD,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

<class 'str'>


,app_id,chart,date,country,rank
0,766894692,free,2014-03-01,United States,1
1,479516143,paid,2014-03-01,United States,1
2,529479190,grossing,2014-03-01,United States,1
3,826459257,free,2014-03-01,United States,2
4,667362389,paid,2014-03-01,United States,2
5,553834731,grossing,2014-03-01,United States,2
6,808176012,free,2014-03-01,United States,3
7,791341471,paid,2014-03-01,United States,3
8,667728512,grossing,2014-03-01,United States,3
9,808032599,free,2014-03-01,United States,4


# APPANNIEHISTORICALEVENTS

Make sure that date is of the format `2014-03-01`. See this [Google Notebook](http://nbviewer.ipython.org/github/mobileinnovationgroup/data_requests/blob/master/main/Google%20Data%20Request.ipynb) for examples of how to do this conversion correctly.

In [14]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPANNIEHISTORICALEVENTS(file_in,out_base,compress=1):
    headers = ['event_type', 'app_id', 'version', 'date']
    out_filename_ios = out_base.format('APPANNIEHISTORICALEVENTS_IOS')
    out_filename_google = out_base.format('APPANNIEHISTORICALEVENTS_GOOGLE')
    with gzip.open(out_filename_ios,'wt',compress) as out_file_ios:
        with gzip.open(out_filename_google,'wt',compress) as out_file_google:
            csv_writer_ios = csv.writer(out_file_ios)
            csv_writer_google = csv.writer(out_file_google)
            csv_writer_ios.writerow(headers)
            csv_writer_google.writerow(headers)
            for json in iter_json_gzip(file_in):
                if 'version' in json.keys():   #some data entries do not have the version data
                    version = json['version']
                else:
                    version = None
                data = [json['event_type'], json['app_id'], version, json['date']]
                if json['store'] == 'ios':
                    csv_writer_ios.writerow(data)
                else:
                    csv_writer_google.writerow(data)
    return [out_filename_ios, out_filename_google]

In [15]:
df_debug = None
if DEBUG:
    csvs = APPANNIEHISTORICALEVENTS(RAW_FILE_HISTORICALEVENTS,OUT_BASE,COMPRESS_LEVEL)
    df_debug_google = read_csv(csvs[1],compression='gzip',header=0)
    df_debug_ios = read_csv(csvs[0],compression='gzip',header=0)
df_debug_ios

,event_type,app_id,version,date
0,version_change,281704574,4.1.1 (iphone os 3.0 tested),2010-02-11
1,initial_release,362348516,NaN,2010-03-31
2,version_change,281704574,4.1.2 (iphone os 3.0 tested),2010-02-16
3,version_change,362348516,1.1,2010-04-28
4,initial_release,520777858,NaN,2012-05-15
5,initial_release,717142133,NaN,2013-10-17
6,version_change,281704574,4.2.1,2010-03-10
7,version_change,362348516,1.2,2010-06-03
8,version_change,520777858,1.02,2012-05-23
9,version_change,362348516,1.3,2010-07-17


In [16]:
df_debug_google

,event_type,app_id,version,date
0,version_change,com.gau.go.launcher.lwp.core,1.0,2014-03-28
1,initial_release,air.com.dpflashes.clearvision3,NaN,2014-01-17
2,version_change,com.azumio.instantheartrate.full,2.5.1,2012-01-17
3,initial_release,com.gau.go.launcher.lwp.core,NaN,2014-03-28
4,version_change,air.com.dpflashes.clearvision3,1.0.2,2014-01-18
5,version_change,com.azumio.instantheartrate.full,2.5.2,2012-01-20
6,version_change,air.com.dpflashes.clearvision3,1.0.3,2014-01-27
7,version_change,com.azumio.instantheartrate.full,2.5.3,2012-01-24
8,initial_release,com.mobage.ww.a956.MARVEL_Card_Battle_Heroes_A...,NaN,2012-10-18
9,version_change,com.mobage.ww.a956.MARVEL_Card_Battle_Heroes_A...,1.0.1,2012-10-26


# APPANNIEHISTORICALRANKINGS

In [17]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def APPANNIEHISTORICALRANKINGS(file_in,out_base,compress=1):
    headers = ['category', 'country', 'app_id', 'rank', 'date']
    out_filename_ios = out_base.format('APPANNIEHISTORICALRANKINGS_IOS')
    out_filename_google = out_base.format('APPANNIEHISTORICALRANKINGS_GOOGLE')
    with gzip.open(out_filename_ios,'wt',compress) as out_file_ios:
        with gzip.open(out_filename_google,'wt',compress) as out_file_google:
            csv_writer_ios = csv.writer(out_file_ios)
            csv_writer_google = csv.writer(out_file_google)
            csv_writer_ios.writerow(headers)
            csv_writer_google.writerow(headers)
            for json in iter_json_gzip(file_in):
                data = [json['category'], json['country'], json['app_id'], json['rank'], json['date']]
                if json['store'] == 'ios':
                    csv_writer_ios.writerow(data)
                else:
                    csv_writer_google.writerow(data)
    return [out_filename_ios, out_filename_google]

In [18]:
df_debug = None
if DEBUG:
    csvs = APPANNIEHISTORICALRANKINGS(RAW_FILE_HISTORICALRANKINGS,OUT_BASE,COMPRESS_LEVEL)
    df_debug_google = read_csv(csvs[1],compression='gzip',header=0)
    df_debug_ios = read_csv(csvs[0],compression='gzip',header=0)
df_debug_ios

,category,country,app_id,rank,date
0,overall,united states,717142133,425,2013-10-17
1,overall,united states,717142133,233,2013-10-18
2,overall,united states,717142133,118,2013-10-19
3,overall,united states,717142133,81,2013-10-20
4,overall,united states,717142133,113,2013-10-21
5,overall,united states,717142133,150,2013-10-22
6,overall,united states,717142133,177,2013-10-23
7,overall,united states,717142133,244,2013-10-24
8,overall,united states,717142133,327,2013-10-25
9,overall,united states,717142133,321,2013-10-26


In [19]:
df_debug_google

,category,country,app_id,rank,date
0,live wallpaper,united states,com.gau.go.launcher.lwp.core,383,2014-04-02
1,overall,united states,air.com.dpflashes.clearvision3,505,2014-02-01
2,live wallpaper,united states,com.gau.go.launcher.lwp.core,167,2014-04-03
3,live wallpaper,united states,com.gau.go.launcher.lwp.core,135,2014-04-04
4,live wallpaper,united states,com.gau.go.launcher.lwp.core,135,2014-04-05
5,overall,united states,air.com.dpflashes.clearvision3,468,2014-02-02
6,live wallpaper,united states,com.gau.go.launcher.lwp.core,76,2014-04-06
7,overall,united states,air.com.dpflashes.clearvision3,416,2014-02-03
8,live wallpaper,united states,com.gau.go.launcher.lwp.core,60,2014-04-07
9,overall,united states,air.com.dpflashes.clearvision3,375,2014-02-04
